# 主旨
使用二元樹CRR計算歐式&美式重設選擇權
1. 歐式
2. 美式

# 載入套件

In [1]:
import numpy as np 
from scipy.stats import norm

# 第一部份 歐式重設選擇權

## 1-1 歐式買權

In [2]:
def eu_reset_call_price( S0:float, K:float, H:float, B:float, r:float, T:float, n:int, sigma:float ) -> np.float64 :
    '''
    功能 : 計算歐式重設買權價格
    輸入 : 
        1. S0 : 期初股價，type=float
        2. K : 原先的履約價，type=float
        3. H : 重設界限，H<S0 & H<K，type=float
        4. B : 重設後的履約價，B<S0 & B<K，type=float
        5. r : 無風險利率，type=float
        6. T : 到期期限(單位:年)，type=float
        7. n : 期數，type=int
        8. sigma : 波動度，type=float
    輸出 :
        call_price : 歐式重設買權價格，type=np.float64
    '''
    # 設定參數
    dt = T / n  # 每一期間隔時間
    u = np.exp( sigma * np.sqrt(dt) )  # 漲幅
    d = 1 / u  # 跌幅
    prob = ( np.exp(r*dt) - d ) / (u - d)  # 股價上漲的風險中立機率
    discount_factor = np.exp(-r*dt)  # 折現因子
    
    # 計算買權期末價格
    ST = S0 * u**n  # 期末股價(一直上漲的情境)
    option_value = [ [0]*2 for _ in range(n+1) ]  # 建構(n+1)*2的list，期末(t=n)共有t+1=n+1個情境(價格)
                                                  # option_value[i][0] = 存放未曾觸碰到重設界線的選擇權價值(履約價=K)
                                                  # option_value[i][1] = 存放曾經觸碰到重設界線的選擇權價值(履約價=B)
    for i in range(n+1) :  # i控制情境(股價下跌次數)
                           # 計算方式 : 從下跌次數i=0的ST開始計算(一直上漲的情境)，一直計算到下跌次數i=n的ST(一直下跌的情境)
        option_value[i][0] = max( ST - K, 0 )  # 未曾觸碰到重設界線的選擇權價值
        option_value[i][1] = max( ST - B, 0 )  # 曾經觸碰到重設界線的選擇權價值
        ST *= d * d  # 計算下個迴圈的ST(ST=第n期上漲股價，ST*d=第n-1期股價，ST*d*d=第n期下跌股價)
        
    # 向前歸納計算買權價格
    # 計算完第t-1期的買權價值後，存放在第t期的位置(覆蓋原本的value)
    for t in range(n-1, -1, -1) :  # 從第n-1期向前歸納至第0期，每次往前1期
        St = S0 * u**t  # 計算第t期下跌次數i=0的股價(一直上漲的情境)
        for i in range(t+1) : # i控制股價下跌次數(第t期共有t+1個情境)
            if St <= H :  # Case1 : 當期股價<=重設界線
                option_value[i][0] = 0  # Case1下useless，填任意數值都可
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
            elif  St > H and St*d <= H :  # Case2 : 當前股價>重設股價&下一期下跌股價<=重設股價 
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][1]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
            else :  # Case3 : 下一期下跌股價>重設股價
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][0]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
            St *= d * d
    call_price = option_value[0][0]
    return call_price

eu_reset_call_price( S0=100, K=100, H=80, B=80, r=0.05, T=1.0, n=3, sigma=0.2 )

12.02302393894727

## 1-2 歐式賣權

In [3]:
def eu_reset_put_price( S0:float, K:float, H:float, B:float, r:float, T:float, n:int, sigma:float ) -> np.float64 :
    '''
    功能 : 計算歐式重設賣權價格
    輸入 : 
        1. S0 : 期初股價，type=float
        2. K : 原先的履約價，type=float
        3. H : 重設界限，H>S0 & H>K，type=float
        4. B : 重設後的履約價，B>S0 & B>K，type=float
        5. r : 無風險利率，type=float
        6. T : 到期期限(單位:年)，type=float
        7. n : 期數，type=int
        8. sigma : 波動度，type=float
    輸出 :
        put_price : 歐式重設賣權價格，type=np.float64
    '''
    # 設定參數
    dt = T / n  # 每一期間隔時間
    u = np.exp( sigma * np.sqrt(dt) )  # 漲幅
    d = 1 / u  # 跌幅
    prob = ( np.exp(r*dt) - d ) / (u - d)  # 股價上漲的風險中立機率
    discount_factor = np.exp(-r*dt)  # 折現因子
    
    # 計算賣權期末價格
    ST = S0 * u**n  # 期末股價(一直上漲的情境)
    option_value = [ [0]*2 for _ in range(n+1) ]  # 建構(n+1)*2的list，期末(t=n)共有t+1=n+1個情境(價格)
                                                  # option_value[i][0] = 存放未曾觸碰到重設界線的選擇權價值(履約價=K)
                                                  # option_value[i][1] = 存放曾經觸碰到重設界線的選擇權價值(履約價=B)
    for i in range(n+1) :  # i控制情境(股價下跌次數)
                           # 計算方式 : 從下跌次數i=0的ST開始計算(一直上漲的情境)，一直計算到下跌次數i=n的ST(一直下跌的情境)
        option_value[i][0] = max( K - ST, 0 )  # 未曾觸碰到重設界線的選擇權價值
        option_value[i][1] = max( B - ST, 0 )  # 曾經觸碰到重設界線的選擇權價值
        ST *= d * d  # 計算下個迴圈的ST(ST=第n期上漲股價，ST*d=第n-1期股價，ST*d*d=第n期下跌股價)
        
    # 向前歸納計算賣權價格
    # 計算完第t-1期的賣權價值後，存放在第t期的位置(覆蓋原本的value)
    for t in range(n-1, -1, -1) :  # 從第n-1期向前歸納至第0期，每次往前1期
        St = S0 * u**t  # 計算第t期下跌次數i=0的股價(一直上漲的情境)
        for i in range(t+1) : # i控制股價下跌次數(第t期共有t+1個情境)
            if St >= H :  # Case1 : 當期股價>=重設界線
                option_value[i][0] = 0  # Case1下useless，填任意數值都可
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
            elif  St < H and St*u >= H :  # Case2 : 當前股價<重設股價&下一期上漲股價>=重設股價 
                option_value[i][0] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][0]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
            else :  # Case3 : 當前&下一期上漲股價<重設股價
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][0]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
            St *= d * d
    put_price = option_value[0][0]
    return put_price

eu_reset_put_price( S0=100, K=100, H=110, B=110, r=0.05, T=1.0, n=3, sigma=0.2 )

7.243429736969338

# 第二部份 美式重設選擇權

## 2-1 美式買權

In [4]:
def us_reset_call_price( S0:float, K:float, H:float, B:float, r:float, T:float, n:int, sigma:float ) -> np.float64 :
    '''
    功能 : 計算美式重設買權價格
    輸入 : 
        1. S0 : 期初股價，type=float
        2. K : 原先的履約價，type=float
        3. H : 重設界限，H<S0 & H<K，type=float
        4. B : 重設後的履約價，B<S0 & B<K，type=float
        5. r : 無風險利率，type=float
        6. T : 到期期限(單位:年)，type=float
        7. n : 期數，type=int
        8. sigma : 波動度，type=float
    輸出 :
        call_price : 美式重設買權價格，type=np.float64
    '''
    # 設定參數
    dt = T / n  # 每一期間隔時間
    u = np.exp( sigma * np.sqrt(dt) )  # 漲幅
    d = 1 / u  # 跌幅
    prob = ( np.exp(r*dt) - d ) / (u - d)  # 股價上漲的風險中立機率
    discount_factor = np.exp(-r*dt)  # 折現因子
    
    # 計算買權期末價格
    ST = S0 * u**n  # 期末股價(一直上漲的情境)
    option_value = [ [0]*2 for _ in range(n+1) ]  # 建構(n+1)*2的list，期末(t=n)共有t+1=n+1個情境(價格)
                                                  # option_value[i][0] = 存放未曾觸碰到重設界線的選擇權價值(履約價=K)
                                                  # option_value[i][1] = 存放曾經觸碰到重設界線的選擇權價值(履約價=B)
    for i in range(n+1) :  # i控制情境(股價下跌次數)
                           # 計算方式 : 從下跌次數i=0的ST開始計算(一直上漲的情境)，一直計算到下跌次數i=n的ST(一直下跌的情境)
        option_value[i][0] = max( ST - K, 0 )  # 未曾觸碰到重設界線的選擇權價值
        option_value[i][1] = max( ST - B, 0 )  # 曾經觸碰到重設界線的選擇權價值
        ST *= d * d  # 計算下個迴圈的ST(ST=第n期上漲股價，ST*d=第n-1期股價，ST*d*d=第n期下跌股價)
        
    # 向前歸納計算買權價格
    # 計算完第t-1期的買權價值後，存放在第t期的位置(覆蓋原本的value)
    exercise_value = [ [0]*2 for _ in range(n+1) ]  # 建構(n+1)*2的list，期末(t=n)共有t+1=n+1個情境(價格)
                                                    # exercise_value[i][0] = 存放未曾觸碰到重設界線的履約價值(履約價=K)
                                                    # exercise_value[i][1] = 存放曾經觸碰到重設界線的履約價值(履約價=B)
    for t in range(n-1, -1, -1) :  # 從第n-1期向前歸納至第0期，每次往前1期
        St = S0 * u**t  # 計算第t期下跌次數i=0的股價(一直上漲的情境)
        for i in range(t+1) : # i控制股價下跌次數(第t期共有t+1個情境)
            if St <= H :  # Case1 : 當期股價<=重設界線
                option_value[i][0] = 0  
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
                exercise_value[i][0] = 0  
                exercise_value[i][1] = St - B
                option_value[i][0] = 0  
                option_value[i][1] = max( option_value[i][1], exercise_value[i][1] )
            elif  St > H and St*d <= H :  # Case2 : 當前股價>重設股價&下一期下跌股價<=重設股價 
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][1]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
                exercise_value[i][0] = St - K
                exercise_value[i][1] = St - B
                option_value[i][0] = max( option_value[i][0], exercise_value[i][0] )
                option_value[i][1] = max( option_value[i][1], exercise_value[i][1] )
            else :  # Case3 : 下一期下跌股價>重設股價
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][0]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
                exercise_value[i][0] = St - K
                exercise_value[i][1] = St - B
                option_value[i][0] = max( option_value[i][0], exercise_value[i][0] )
                option_value[i][1] = max( option_value[i][1], exercise_value[i][1] )
            St *= d * d
    call_price = option_value[0][0]
    return call_price

us_reset_call_price( S0=100, K=100, H=80, B=80, r=0.05, T=1.0, n=3, sigma=0.2 )

12.02302393894727

## 2-2 美式買權

In [5]:
def us_reset_put_price( S0:float, K:float, H:float, B:float, r:float, T:float, n:int, sigma:float ) -> np.float64 :
    '''
    功能 : 計算美式重設賣權價格
    輸入 : 
        1. S0 : 期初股價，type=float
        2. K : 原先的履約價，type=float
        3. H : 重設界限，H<S0 & H<K，type=float
        4. B : 重設後的履約價，B<S0 & B<K，type=float
        5. r : 無風險利率，type=float
        6. T : 到期期限(單位:年)，type=float
        7. n : 期數，type=int
        8. sigma : 波動度，type=float
    輸出 :
        put_price : 美式重設賣權價格，type=np.float64
    '''
    # 設定參數
    dt = T / n  # 每一期間隔時間
    u = np.exp( sigma * np.sqrt(dt) )  # 漲幅
    d = 1 / u  # 跌幅
    prob = ( np.exp(r*dt) - d ) / (u - d)  # 股價上漲的風險中立機率
    discount_factor = np.exp(-r*dt)  # 折現因子
    
    # 計算賣權期末價格
    ST = S0 * u**n  # 期末股價(一直上漲的情境)
    option_value = [ [0]*2 for _ in range(n+1) ]  # 建構(n+1)*2的list，期末(t=n)共有t+1=n+1個情境(價格)
                                                  # option_value[i][0] = 存放未曾觸碰到重設界線的選擇權價值(履約價=K)
                                                  # option_value[i][1] = 存放曾經觸碰到重設界線的選擇權價值(履約價=B)
    for i in range(n+1) :  # i控制情境(股價下跌次數)
                           # 計算方式 : 從下跌次數i=0的ST開始計算(一直上漲的情境)，一直計算到下跌次數i=n的ST(一直下跌的情境)
        option_value[i][0] = max( K - ST, 0 )  # 未曾觸碰到重設界線的選擇權價值
        option_value[i][1] = max( B - ST, 0 )  # 曾經觸碰到重設界線的選擇權價值
        ST *= d * d  # 計算下個迴圈的ST(ST=第n期上漲股價，ST*d=第n-1期股價，ST*d*d=第n期下跌股價)
        
    # 向前歸納計算賣權價格
    # 計算完第t-1期的賣權價值後，存放在第t期的位置(覆蓋原本的value)
    exercise_value = [ [0]*2 for _ in range(n+1) ]  # 建構(n+1)*2的list，期末(t=n)共有t+1=n+1個情境(價格)
                                                    # exercise_value[i][0] = 存放未曾觸碰到重設界線的履約價值(履約價=K)
                                                    # exercise_value[i][1] = 存放曾經觸碰到重設界線的履約價值(履約價=B)
    for t in range(n-1, -1, -1) :  # 從第n-1期向前歸納至第0期，每次往前1期
        St = S0 * u**t  # 計算第t期下跌次數i=0的股價(一直上漲的情境)
        for i in range(t+1) : # i控制股價下跌次數(第t期共有t+1個情境)
            if St >= H :  # Case1 : 當期股價>=重設界線
                option_value[i][0] = 0  
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
                exercise_value[i][0] = 0  
                exercise_value[i][1] = B - St
                option_value[i][0] = 0  
                option_value[i][1] = max( option_value[i][1], exercise_value[i][1] )
            elif  St < H and St*u >= H :  # Case2 : 當前股價<重設股價&下一期上漲股價>=重設股價 
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][1]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
                exercise_value[i][0] = K - St
                exercise_value[i][1] = B - St
                option_value[i][0] = max( option_value[i][0], exercise_value[i][0] )
                option_value[i][1] = max( option_value[i][1], exercise_value[i][1] )
            else :  # Case3 : 當前&下一期上漲股價<重設股價
                option_value[i][0] = discount_factor * ( option_value[i][0]*prob + option_value[i+1][0]*(1-prob) )
                option_value[i][1] = discount_factor * ( option_value[i][1]*prob + option_value[i+1][1]*(1-prob) )
                exercise_value[i][0] = K - St
                exercise_value[i][1] = B - St
                option_value[i][0] = max( option_value[i][0], exercise_value[i][0] )
                option_value[i][1] = max( option_value[i][1], exercise_value[i][1] )
            St *= d * d
    put_price = option_value[0][0]
    return put_price

us_reset_put_price( S0=100, K=100, H=110, B=110, r=0.05, T=1.0, n=3, sigma=0.2 )

9.379835004059139